In [3]:
from dash import Dash, html, dcc, callback, Output, Input, State
from dash_bootstrap_templates import load_figure_template
from dash import dash_table
import dash_bootstrap_components as dbc
import plotly_express as px
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

ID	Name	Sex	Age	Height	Weight	Team	NOC	Games	Year	Season	City	Sport	Event	Medal

In [32]:
athletes_events = pd.read_csv("../Uppgifter/Filer/athlete_events.csv")
athletes_events

medals = athletes_events.groupby(["NOC", "Medal", "Sport"], as_index=False).size().rename(columns={"size": "Medal_count"})
medals
df = medals.query("Sport=='Judo'")
df = df.groupby("NOC", as_index=False)["Medal_count"].sum().sort_values("Medal_count", ascending=False)
countries = list(df.head(5)["NOC"])
medals[medals["NOC"].isin(countries)]

age_groups = athletes_events.groupby(["Age", "Sport"], as_index=False).size().rename(columns={"size": "total"})



,Age,Sport,total
0,10.0,Gymnastics,1
1,11.0,Figure Skating,7
2,11.0,Gymnastics,1
3,11.0,Rowing,1
4,11.0,Swimming,4
...,...,...,...
1829,81.0,Art Competitions,2
1830,84.0,Art Competitions,1
1831,88.0,Art Competitions,3
1832,96.0,Art Competitions,1


In [55]:
athletes_events = pd.read_csv("../Uppgifter/Filer/athlete_events.csv") 
load_figure_template("darkly")

app = Dash(
    __name__, 
    external_stylesheets=[dbc.themes.DARKLY],
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Japan OS", className='text-center text-primary mx-3'),
            html.Br(),
            dcc.DatePickerRange(id='date_picker', className='mx-auto mb-4')
        ], width=12, style={'text-align': 'center'})
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id="single_dropdown", multi=False, searchable=False, className='mb-2', value=['Judo', 'Basketball', 'Football'],
                options=sorted([sport for sport in athletes_events["Sport"].unique()]),
                style={'color': '#333'}
            ),
            dcc.Graph(id="Medal_graph",
                figure={})
        ], xs=10, sm=10, md=10, lg=4),
        
    ], justify="start"),

    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id="age_dropdown", multi=False, searchable=False, className='mb-2', value=[],
                options=sorted([sport for sport in athletes_events["Sport"].unique()]),
                style={'color': '#333'}
            ),
            dcc.Graph(id="Age_graph",
                figure={})
        ], xs=10, sm=10, md=10, lg=4),       
    ], justify="start"),

], fluid=True)


@callback(
    Output("Age_graph", "figure"),
    Input("age_dropdown", "value")
)

def age_groups(selected_age):
     filtered_df = athletes_events[athletes_events["Age"] == selected_age] 
     age_groups = filtered_df.groupby(["Age", "Sport"], as_index=False).size().rename(columns={"size": "total"})
     age_groups = age_groups.sort_values(by=["Age", "Sport"])
     figure = px.bar(age_groups, x="Age", y="Sport", color="Age")
     return figure


@callback(
    Output("Medal_graph", "figure"),
    Input("single_dropdown", "value"),   
)

def top_5_countries(selected_sport):
     df = medals.query("Sport==@selected_sport")
     grouped_df = df.groupby("NOC", as_index=False)["Medal_count"].sum()
     top_5_countries = list(grouped_df.sort_values("Medal_count", ascending=False).head(5)["NOC"])
     df = df[df["NOC"].isin(top_5_countries)]
     fig = px.sunburst(df, path=['NOC', 'Medal'], values='Medal_count', color='NOC', hover_name='Medal_count') #hover_data=[False, False, False, False])#{'NOC': False, 'Medal': False, 'Medal_count': False, 'parent': False, 'id': False})
     return fig


app.run(debug=True, jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_chart_types.py:373, in bar(
    data_frame=Empty DataFrame
Columns: [Age, Sport, total]
Index: [],
    x='Age',
    y='People',
    color='Age',
    pattern_shape=None,
    facet_row=None,
    facet_col=None,
    facet_col_wrap=0,
    facet_row_spacing=None,
    facet_col_spacing=None,
    hover_name=None,
    hover_data=None,
    custom_data=None,
    text=None,
    base=None,
    error_x=None,
    error_x_minus=None,
    error_y=None,
    error_y_minus=None,
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels=None,
    color_discrete_sequence=None,
    color_discrete_map=None,
    color_continuous_scale=None,
    pattern_shape_sequence=None,
    pattern_shape_map=None,
    range_color=None,
    color_contin